In [ ]:
#Import packages
import pandas as pd
import os
import numpy as np
from datetime import date
today = str(date.today())
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999

In [ ]:
#Set path variables and read in data set
user = os.path.expanduser('~')
home = os.path.join(user, 'Box/LS_Folders/CANDLab_LS/TraumaData')
data = os.path.join(home, 'RI_Downloads/UCLARIADULT52218_DATA_2019-07-18_0919.csv')    
dset = pd.read_csv(data, header = 0)
dset = dset.replace('999',np.nan)
dset = dset.replace(999,np.nan)

In [ ]:
da = os.path.join(home, 'RI_Downloads/UCLARIADULT52218_DATA_2019-08-20_1259.csv')
d = pd.read_csv(da, header = 0)
d = d.replace('999',np.nan)
d = d.replace(999,np.nan)
d_clean = d.dropna(axis=1, how='all')
#d_clean.to_csv(os.path.join(home, 'RI_Downloads/8.20.19_clean.csv'))



In [185]:
d.columns

Index(['ucla_a_id', 'ucla_a_age', 'ucla_a_admin', 'ucla_a_date', 'ucla_a_val',
       'ucla_a_valsca', 'ucla_a_a', 'ucla_a_a_1', 'ucla_a_a_2', 'ucla_a_a_3',
       ...
       'ucla_a_e_e28_viccod___8', 'ucla_a_e_e28_viccod___9',
       'ucla_a_e_e28_viccod___10', 'ucla_a_e_e28_viccod___11',
       'ucla_a_e_e28_viccod___12', 'ucla_a_e_e28_viccod___13',
       'ucla_a_e_e28_viccod___14', 'ucla_a_e_e28_viccod___15',
       'ucla_a_e_e28_viccod___16', 'ucla_a_e_e28_viccod___17'],
      dtype='object', length=16384)

In [184]:
d_clean.columns

Index(['ucla_a_id', 'ucla_a_age', 'ucla_a_admin', 'ucla_a_date', 'ucla_a_val',
       'ucla_a_valsca', 'ucla_a_a', 'ucla_a_a_1', 'ucla_a_a_2', 'ucla_a_a_3',
       ...
       'ucla_a_e_e28_viccod___8', 'ucla_a_e_e28_viccod___9',
       'ucla_a_e_e28_viccod___10', 'ucla_a_e_e28_viccod___11',
       'ucla_a_e_e28_viccod___12', 'ucla_a_e_e28_viccod___13',
       'ucla_a_e_e28_viccod___14', 'ucla_a_e_e28_viccod___15',
       'ucla_a_e_e28_viccod___16', 'ucla_a_e_e28_viccod___17'],
      dtype='object', length=14001)

In [ ]:
empty_cols = []
def get_cols(df, col_num):
    col = df.iloc[:,col_num].dropna().astype(int)
    if col.sum() < 1 :
        empty_cols.append(col_num)
    return col.sum()

In [ ]:
for z in range(4,len(d_clean.columns)):
    #print(x)
    get_cols(d_clean, z)

empty_cols

In [186]:
d_clean_empty = d_clean.drop(d_clean.columns[empty_cols], axis=1).dropna(axis=1, how='all')
d_clean_empty.columns#.to_csv(os.path.join(home, 'RI_Downloads/{}_clean.csv'))

Index(['ucla_a_id', 'ucla_a_age', 'ucla_a_admin', 'ucla_a_date',
       'ucla_a_valsca', 'ucla_a_a', 'ucla_a_a_1', 'ucla_a_a_2', 'ucla_a_a_3',
       'ucla_a_a_ages___4',
       ...
       'ucla_a_e_e6_avgrxn', 'ucla_a_e_e6_viccod___21',
       'ucla_a_e_e6_percod___21', 'ucla_a_e_e6_percod___23',
       'ucla_a_e_e6_icthr', 'ucla_a_e_e6_pq2', 'ucla_a_e_e6_crita',
       'ucla_a_e_e6_critacod___5', 'ucla_a_e_e6_critacod___6',
       'ucla_a_e_e6_critadwl___1'],
      dtype='object', length=2189)

In [ ]:
#Function to catch empty columns; create list
empty_cols = []
def get_cols(df, col_num):
    col = df.iloc[:,col_num].dropna().astype(int)
    if col.sum() < 1 :
        empty_cols.append(col_num)
    return col.sum()

f

In [ ]:
#Drop empty columns
dset_cleaned = dset.drop(dset.columns[empty_cols], axis=1).dropna(axis=1, how='all')

#Create threat and dep variables
thr_cols = [col for col in dset_cleaned.columns if 'icthr' in col]
dep_cols = [col for col in dset_cleaned.columns if 'icdep' in col]
thr_cols.append('ucla_a_id')
dep_cols.append('ucla_a_id')

In [ ]:
df_thr = dset[thr_cols]
df_dep = dset[dep_cols]
df_thr['num_thr_ev'] = df_thr.sum(axis=1)
df_dep['num_dep_ev'] = df_dep.sum(axis=1)
final_summed_thr = df_thr[['ucla_a_id','num_thr_ev']]
final_summed_dep = df_dep[['ucla_a_id','num_dep_ev']]
final_summed=pd.merge(final_summed_thr, final_summed_dep, on='ucla_a_id', how='outer')

In [ ]:
#Create thr/dep by age function
pairs=[]
def create_age_exp_pairs(df, typ_exp):
    criteria1 = df.columns.str.contains('age')
    criteria2 = df.columns.str.contains('ic{}'.format(typ_exp))
    criteria_all = criteria1 | criteria2
    cols = df.columns[criteria_all]
    new_df=df[cols]
    return new_df

#Create thr/dep by age data frames
thr_age_df = create_age_exp_pairs(dset_cleaned, 'thr').dropna(axis=1, how='all')
dep_age_df = create_age_exp_pairs(dset_cleaned, 'dep').dropna(axis=1, how='all')


In [ ]:
#Set up loops to create new dfs by age

events = ['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','x','y']
numoccs = range(1,32) #number of occurrences -- up to 31 for each event

In [ ]:
#Empty lists to append threat/dep scores
thr_df = []
dep_df = []
all_df = []

#Define function to count number of endorsements of threat or deprivation
#Where e is events and x is numoccs
def score_num(dset, e, x, thde): 
    data = dset.iloc[i]
    try:
        if data['ucla_a_{}_e{}_{}'.format(e, x, thde)] == int('1'):
            if thde == 'icthr':
                sub = str(data['ucla_a_id'])
                item = str(data['ucla_a_{}_e{}_{}'.format(e, x, thde)])
                sev = str(data['ucla_a_{}_e{}_avgsev'.format(e, x)])
                age = str(data['ucla_a_{}_e{}_age'.format(e, x)])
                thr_df.append([sub, age, item, sev])
            elif thde == 'icdep':
                sub = str(data['ucla_a_id'])
                item = str(data['ucla_a_{}_e{}_{}'.format(e, x, thde)])
                sev = str(data['ucla_a_{}_e{}_avgsev'.format(e, x)])
                age = str(data['ucla_a_{}_e{}_age'.format(e, x)])
                dep_df.append([sub, age, item, sev])
            elif thde == 'all':
                pass
    except:
        pass

In [ ]:
d = []
data=dset_cleaned.iloc[3]
x = str(data['ucla_a_id'])
d.append(x)
d

In [ ]:
#Perform actual counts
for i in range(0,len(dset_cleaned)):
    for e in events:
        for x in numoccs:
            score_num(dset_cleaned,e,x,'icthr')
            score_num(dset_cleaned,e,x,'icdep')


In [ ]:
thr_data = pd.DataFrame(thr_df)
thr_data

In [ ]:
threat_df1 = pd.DataFrame(thr_df).groupby(0).sum().reset_index()
threat_df= threat_df1.rename(columns={0:'ucla_a_id', 1:"num_threat"})

depriv_df1 = pd.DataFrame(dep_df).groupby(0).sum().reset_index()
depriv_df = depriv_df1.rename(columns={0:'ucla_a_id', 1:"num_dep"})

merged_df = pd.merge(threat_df, depriv_df, on='ucla_a_id', how='outer')

finaldf = pd.merge(merged_df, dset, on='ucla_a_id', how='outer')

In [ ]:
merged_df.to_csv(os.path.join(home, 'LS_scored_traumaData_'+today+'.csv'))